In [ ]:
# libraries
import numpy as np
import torch
import torch.nn.functional as F
import cv2
import time
import matplotlib.pyplot as plt
from sklearn.metrics import recall_score, f1_score


In [ ]:
# device use CUDA
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# hyperparameters
random_seed = 1
learning_rate = 0.001
num_epochs = 100
batch_size = 256

# architecture
num_classes = 2


In [ ]:
# import ADNI dataset


In [ ]:
# split data into X and y with split of 70/30
train_dataset = ?
test_dataset = ?
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size,  shuffle=True)
test_loader = DataLoader(dataset=test_dataset,  batch_size=batch_size,  shuffle=False)


In [ ]:
# check dataset
for images, labels in train_loader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    break

In [ ]:
# visualizing neuroimages


In [ ]:
# pre-processing neuroimages


In [ ]:
# CNN model
class CNN_model(torch.nn.Module):
    def __init__(self, num_classes):
        super(CNN_model, self).__init__()

        self.conv_1 = torch.nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, stride=1, padding=0)

        self.maxpool_1 = torch.nn.MaxPool2d(2, 2)


        self.conv_2 = torch.nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=0) 

        self.maxpool_2 = torch.nn.MaxPool2d(2, 2)


        self.conv_3 = torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, stride=1, padding=0) 

        self.maxpool_3 = torch.nn.MaxPool2d(2, 2)


        self.conv_4 = torch.nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5, stride=1, padding=0) 

        self.maxpool_4 = torch.nn.MaxPool2d(2, 2)


        self.fc1 = torch.nn.Linear(?, 1024)

        self.fc2 = torch.nn.Linear(1024, num_classes)

    def forward(self, x):
        out = F.relu(self.conv_1(x))
        out = self.maxpool_1(out)

        out = F.relu(self.conv_2(x))
        out = self.maxpool_2(out)

        out = F.relu(self.conv_3(x))
        out = self.maxpool_3(out)

        out = F.relu(self.conv_4(x))
        out = self.maxpool_4(out)

        out = out.view(-1, ?)
        out = F.relu(self.fc1(out))
        out = self.fc2(out)

        return out
        

In [ ]:
# training CNN model
torch.manual_seed(random_seed)

model = CNN_model(num_classes=num_classes).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

criterion = torch.nn.CrossEntropyLoss()

start_time = time.time()

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for batch_idx, (features, targets) in enumerate(train_loader):
        features, targets = features.to(device), targets.to(device)

        optimizer.zero_grad()
        
        outputs = model(features)

        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if batch_idx % 100 == 99:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, batch_idx + 1, running_loss / 100))
            running_loss = 0.0

print('CNN Training Finished')

In [ ]:
# compute accuracy of model
def compute_accuracy(model, data_loader):
    correct=0
    total=0

    for batch_idx, (features, targets) in enumerate(data_loader):
        features, targets = features.to(device), targets.to(device)

        outputs = model(features)
        _, predicted = torch.max(outputs, 1)

        correct += (predicted == targets).sum().item()
        total += targets.size(0)
    acc = correct/total
    acc = 100 * acc
    return acc

In [ ]:
# compute sensitivity
def compute_sensitivity(model, data_loader):
    truePositive=0
    falseNegative=0

    for batch_idx, (features, targets) in enumerate(data_loader):
        features, targets = features.to(device), targets.to(device)

        outputs = model(features)
        _, predicted = torch.max(outputs, 1)

        truePositive += ((predicted == 1) & (targets == 1)).sum().item()
        falseNegative += ((predicted == 0) & (targets == 1)).sum().item()

        sens = truePositive / (truePositive + falseNegative)

        return sens

In [ ]:
# compute specificity
def compute_specificity(model, data_loader):
    trueNegative=0
    falsePositive=0

    for batch_idx, (features, targets) in enumerate(data_loader):
        features, targets = features.to(device), targets.to(device)

        outputs = model(features)
        _, predicted = torch.max(outputs, 1)

        trueNegative += ((predicted == 0) & (targets == 0)).sum().item()
        falsePositive += ((predicted == 1) & (targets == 0)).sum().item()

        spec = trueNegative / (trueNegative + falsePositive)

        return spec

In [ ]:
# compute f1-score
def compute_f1_score(model, data_loader):

    return 0

In [ ]:
print('Test accuracy: %.2f%%' % (compute_accuracy(model, test_loader)))
print('Test sensitivity: %.2f%%' % (compute_sensitivity(model, test_loader)))
print('Test specificity: %.2f%%' % (compute_specificity(model, test_loader)))
print('Test f1-score: %.2f%%' % (compute_f1_score(model, test_loader)))